### Matching

In [1]:
import pandas as pd

In [2]:
df1=pd.read_excel('CSV\\prepared_job_data.xlsx')
df2=pd.read_csv('CSV\\PreparedCV.csv')

In [3]:
df1 = df1.drop('Unnamed: 0', axis=1)

In [4]:
df1.to_csv('CSV\\job_offers.csv', index=False)

In [4]:
df1.to_json('CSV\\Jobs.json', orient='records')

In [5]:
print(df1['skill_required'])


0       software engineering, data development solutio...
1       Broad spectrum technologies (including Blockch...
2       Collaboration, architecture, design, deploymen...
3       Kafka, Java8 (JDK11/17), Spring Cloud Stream, ...
4       Cloudera/Spark/SnowFlake/Databricks/Stratio/Hi...
                              ...                        
1293    pour identifier des anomalies de fonctionnemen...
1294    globale sur le SI, documenter et produire le m...
1295    implémentation chatbot intelligent, outil préd...
1296    Les techniques de webscrapping ou/et parsing d...
1297         méthodologie, questionnaire (questions clés)
Name: skill_required, Length: 1298, dtype: object


In [152]:
print(df2)

                                                   URL  \
0                https://www.linkedin.com/in/talel-kb/   
1         https://www.linkedin.com/in/omar-talbi-sfax/   
2    https://www.linkedin.com/in/hassen-knani-21991...   
3            https://www.linkedin.com/in/safwendammak/   
4    https://www.linkedin.com/in/oumayma-b%C3%A9hi-...   
..                                                 ...   
429  https://www.linkedin.com/in/khaoula-ben-othman...   
430  https://www.linkedin.com/in/farah-abid-989683191/   
431  https://www.linkedin.com/in/achref-cherif-data...   
432          https://www.linkedin.com/in/hamza-samaiy/   
433  https://www.linkedin.com/in/zitouni-amal-286b9...   

                                             job_title                Name  \
0          web developer  software engineering student        Talel Kbaier   
1                              freelance web developer          Omar Talbi   
2                           web developer chez zetabox        Hassen 

In [6]:
df2['skills'].dropna()

0      analyse donnees competences analytiques framew...
1      laravel react javascript php microsoft bot fra...
2      management service client microsoft office ven...
3      angular phpsymfony developpement web developpe...
4                                              mentioned
                             ...                        
429    mongodb rstudio business intelligence bi pyspa...
430    analyse donnees nlp extraction donnees python ...
431    extract transform load etl analytical skills d...
432    developpement web javascript php laravel feuil...
433    microsoft azure machine learning apprentissage...
Name: skills, Length: 434, dtype: object

In [7]:
print(df2['skills'][2])

management service client microsoft office vente gestion projet robotique gestion risques microcontroleurs mysql arduino microsoft word matlab labview solidworks leadership communication gestion d'equipe expressjs programmation javascript bootstrap mongodb reduxjs reactjs nodejs css


In [8]:
print(df1['skill_required'][2])

Collaboration, architecture, design, deployment solutions, system integration, automation, execution support, pipeline deployments, ALM tools, cyber security, SW deployments, automation functional regression tests, continuous integration, environments infrastructures, continuous monitoring, automatic deployment tools (Terraform, Ansible, etc.), cloud architectures (vCloud, Azure/AWS), monitoring tools, OpenShift/DockerHub/Kubernetes/CloudALM, Jira/Confluence, Git, Jenkins (CloudBees), Nexus, SonarHarbour, scripting languages (Groovy, Python, JavaScript, etc.), PostgreSQL, MongoDB, AzureCosmosDB


## preparation for words embedding of jobs dataset

removing stop words from skill_required column

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')




[nltk_data] Downloading package stopwords to C:\Users\MSI
[nltk_data]     GF63\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\MSI
[nltk_data]     GF63\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
stop_words = set(stopwords.words('english'))

def clean(text):
    tokens = word_tokenize(text)

    filtered_tokens = [word for word in tokens if not word.lower() in stop_words]

    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [13]:
df1['skill_required'] = df1['skill_required'].apply(clean)
df2['skills'] = df2['skills'].apply(clean)


In [14]:
print(df1['skill_required'][4])

Cloudera/Spark/SnowFlake/Databricks/Stratio/Hive/Impala/HBase , Spark Streaming/Flink/Storm , API Ecosystems , HDFS , S3 , Kafka , Scala/Python/Java , CI/CD , Google/AWS/Azure


In [15]:
print(df2['skills'][3])

angular phpsymfony developpement web developpement logiciels applications web conception technique methodes agiles gestion projet developpement frontend javascript jquery php wordpress xml json plsql uml html symfony cakephp typescript developpement web backend framework symfony conception frontend git mysql sql twitter bootstrap ajax feuilles style cascade css linux api rest


In [16]:
import ast
#list_from_string = ast.literal_eval("['Python', 'AI', 'Digitallending']")

#print(list_from_string)
def tolist(text):
    try:
        split_skills= ast.literal_eval(text)
        split_skills = [item.split('/') for item in split_skills]
        flat_list = [item.strip() for sublist in split_skills for item in sublist]
        return flat_list 
    except:
        split_skills=text.split(',')
        split_skills = [item.split('/') for item in split_skills]
        flat_list = [item.strip() for sublist in split_skills for item in sublist]
        return flat_list 

        
df1['skill_required'] = df1['skill_required'].apply(tolist)
df2['skills'] = df2['skills'].apply(tolist)


In [17]:
print(df2['skills'][1])

['laravel react javascript php microsoft bot framework java testng selenium testing management programming language objectoriented programming oop unity sql databases microsoft power automate sharepoint microsoft power apps azure functions cloud computing automation agile methodologies microsoft azure microsoft office python programming language nosql']


In [18]:
print(df1['skill_required'][1])

['Broad spectrum technologies ( including Blockchain', 'Big Data', 'Angular ) kinds premise & cloud-based platforms']


dataframe where every skill is a column 

In [19]:
# create a set of all unique skills
all_skills = set([skill for skills_list in df1['skill_required'] for skill in skills_list])

# create a dictionary with keys as skills and values as empty lists
skills_dict = {skill: [] for skill in all_skills}

# fill the dictionary with 1 or 0 depending on whether the skill is present or not in the skills list
for skills_list in df1['skill_required']:
    for skill in all_skills:
        if skill in skills_list:
            skills_dict[skill].append(1)
        else:
            skills_dict[skill].append(0)

# create a new data frame from the dictionary
skills_df = pd.DataFrame.from_dict(skills_dict)


## Cosinie Similarity

word embedding

In [20]:
import gensim.downloader as api
import numpy as np
wv = api.load('word2vec-google-news-300')


In [21]:
import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from gensim.models import Word2Vec
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('english')

from numpy import dot
from numpy.linalg import norm

[nltk_data] Downloading package wordnet to C:\Users\MSI
[nltk_data]     GF63\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\MSI
[nltk_data]     GF63\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [23]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    text=str(text)
    # Remove stopwords from the text
    text = remove_stopwords(text)
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            if token == 'xxxx':
                continue
            result.append(lemmatize_stemming(token))
    
    return result

In [24]:
processed_docs=df1['skill_required'].map(preprocess)

In [25]:
p=df2['skills'].map(preprocess)

In [26]:
print(p[5])

['program', 'languag', 'microsoft', 'excel', 'javascript', 'program', 'program', 'concept', 'html', 'design', 'small', 'busi', 'manag', 'design', 'theori', 'cascad', 'style', 'sheet', 'python', 'program', 'languag', 'adob', 'photoshop', 'experi', 'utilisateur', 'feuill', 'style', 'cascad', 'graphiqu', 'illustr', 'graphism', 'standard', 'bootstrap', 'developp', 'developp', 'frontend', 'frontend', 'develop', 'develop', 'standard', 'user', 'experi', 'access', 'access', 'typographi', 'typographi', 'site', 'adaptatif', 'sass', 'stylesheet', 'languag', 'nodej']


In [27]:
print(processed_docs[13])

['valorará', 'acreditar', 'experiencia', 'profesion', 'buena', 'soft', 'skill', 'buena', 'capacidad', 'comunicativa', 'relación', 'nivel', 'proactivo', 'miedo', 'cambio', 'orientado', 'objetivo', 'buena', 'habilidad', 'ofimática', 'excel', 'powerpoint', 'necesario', 'buen', 'nivel', 'inglé']


In [28]:
print(len(processed_docs))

1298


In [29]:
print(len(p))

434


In [30]:
processed_data=pd.concat([processed_docs,p])

In [31]:
processed_data2=processed_data.map(preprocess)

In [32]:
processed_data=pd.concat([processed_data,processed_data2])

In [33]:
print(len(processed_data))

3464


In [34]:
def word2vec_model():
    w2v_model = Word2Vec(min_count=1,
                     window=3,
                     vector_size=50,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20)
    
    w2v_model.build_vocab(processed_data)
    w2v_model.train(processed_data, total_examples=w2v_model.corpus_count, epochs=300, report_delay=1)
    
    return w2v_model

In [35]:
w2v_model = word2vec_model()
w2v_model.save('word2vec_model_noSW')

In [36]:
emb_vec = w2v_model.wv

In [37]:
emb_vec['python'] # It will return vector representation of the word python

array([ 0.01058971, -1.3089925 ,  0.08116454,  0.22937128,  0.5539384 ,
       -1.1946093 , -0.0154536 ,  0.27638793,  0.19544479,  0.34801197,
        0.715787  ,  0.01268382,  0.15384103,  0.41719112, -0.40052298,
        0.40317827,  0.13914669,  0.89994323, -0.36767638,  1.111186  ,
        0.53451055, -0.28778636,  0.5552984 ,  1.466706  ,  0.72034526,
        0.9204868 ,  0.05575425,  0.6271853 ,  0.05831338, -0.35136902,
       -0.49765384, -1.0936556 , -0.76823455, -0.3996785 , -1.4230335 ,
        0.46341613,  1.2635194 , -0.17408012, -0.7993756 , -0.5798614 ,
       -0.07294805, -0.37899193, -0.4241187 ,  0.31042087, -0.11437657,
       -0.1209499 , -0.24423723, -1.365406  ,  0.41990322, -0.54702914],
      dtype=float32)

## Cosine Similarity

In [38]:
def find_similarity(sen1, sen2, model):
    p_sen1 = preprocess(sen1)
    p_sen2 = preprocess(sen2)
    
    sen_vec1 = np.zeros(50)
    sen_vec2 = np.zeros(50)
    for val in p_sen1:
        sen_vec1 = np.add(sen_vec1, model[val])

    for val in p_sen2:
        sen_vec2 = np.add(sen_vec2, model[val])
    return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))
def find_similarity2(sen1, sen2, model): 
    p_sen1 = preprocess(sen1)
    p_sen2 = preprocess(sen2)
    sen_vec1 = np.zeros(50)
    sen_vec2 = np.zeros(50)
    for val in p_sen1:
        sen_vec1 = np.add(sen_vec1, model[val])

    for val in p_sen2:
        sen_vec2 = np.add(sen_vec2, model[val])
    return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))

In [39]:
find_similarity('JAVA developement','java',emb_vec)

0.8985078152538182

In [40]:
find_similarity('Android, Java', 'Android, Java, Python, Django, Xamarin, Cloud',emb_vec )

0.8711619532458093

In [41]:
find_similarity('Java', 'mobil',emb_vec )

0.4433011472082337

In [42]:
print(df2['skills'][1])

['laravel react javascript php microsoft bot framework java testng selenium testing management programming language objectoriented programming oop unity sql databases microsoft power automate sharepoint microsoft power apps azure functions cloud computing automation agile methodologies microsoft azure microsoft office python programming language nosql']


In [43]:
print(df1['skill_required'][2])

['Collaboration', 'architecture', 'design', 'deployment solutions', 'system integration', 'automation', 'execution support', 'pipeline deployments', 'ALM tools', 'cyber security', 'SW deployments', 'automation functional regression tests', 'continuous integration', 'environments infrastructures', 'continuous monitoring', 'automatic deployment tools ( Terraform', 'Ansible', 'etc . )', 'cloud architectures ( vCloud', 'Azure', 'AWS )', 'monitoring tools', 'OpenShift', 'DockerHub', 'Kubernetes', 'CloudALM', 'Jira', 'Confluence', 'Git', 'Jenkins ( CloudBees )', 'Nexus', 'SonarHarbour', 'scripting languages ( Groovy', 'Python', 'JavaScript', 'etc . )', 'PostgreSQL', 'MongoDB', 'AzureCosmosDB']


In [44]:
find_similarity(str(df2['skills'][1]),str(df1['skill_required'][2]),emb_vec)#need to train the emb_vec on the skills dataset too

0.38975029706314246

In [45]:
def similarity(skill,required,model):
    p_skill=preprocess(skill)
    p_required=preprocess(required)
    result=[]
    for i in range(len(p_required)):
        similarities=[]
        for j in range(len(p_skill)):
            similarities.append(find_similarity2(p_skill[j],p_required[i],model))
        result.append(max(similarities)) 
    result = [0 if pd.isna(x) else x for x in result]    
    if len(result) > 0:
        avg_similarity = sum(result) / len(result)
    else:
        avg_similarity = 0
    return avg_similarity,required,result    
    #we returned the similarity score , the required skills  ,all similiraty for every word alone

In [46]:
similarity('Android, Java, Python, Django', 'Android, Java ',emb_vec )

(1.0, 'Android, Java ', [1.0, 1.0])

In [47]:
similarity('Android, Java', 'Android, Java, Python, Django ',emb_vec )

(0.8594404348701894,
 'Android, Java, Python, Django ',
 [1.0, 1.0, 0.849387563477178, 0.5883741760035798])

In [48]:
similarity(df2['skills'][4],df1['skill_required'][2],emb_vec)#need to train the emb_vec on the skills dataset too

(0.02842758066361597,
 ['Collaboration',
  'architecture',
  'design',
  'deployment solutions',
  'system integration',
  'automation',
  'execution support',
  'pipeline deployments',
  'ALM tools',
  'cyber security',
  'SW deployments',
  'automation functional regression tests',
  'continuous integration',
  'environments infrastructures',
  'continuous monitoring',
  'automatic deployment tools ( Terraform',
  'Ansible',
  'etc . )',
  'cloud architectures ( vCloud',
  'Azure',
  'AWS )',
  'monitoring tools',
  'OpenShift',
  'DockerHub',
  'Kubernetes',
  'CloudALM',
  'Jira',
  'Confluence',
  'Git',
  'Jenkins ( CloudBees )',
  'Nexus',
  'SonarHarbour',
  'scripting languages ( Groovy',
  'Python',
  'JavaScript',
  'etc . )',
  'PostgreSQL',
  'MongoDB',
  'AzureCosmosDB'],
 [0.08617587730384572,
  0.30422849592243745,
  0.11549021976066273,
  -0.02844834319267346,
  0.10827030719726213,
  0.006098416418639263,
  0.13963004369895682,
  0.07327413336630476,
  0.2155511978084

In [49]:
print(df2['skills'][4])
print(df1['skill_required'][2])

['mentioned']
['Collaboration', 'architecture', 'design', 'deployment solutions', 'system integration', 'automation', 'execution support', 'pipeline deployments', 'ALM tools', 'cyber security', 'SW deployments', 'automation functional regression tests', 'continuous integration', 'environments infrastructures', 'continuous monitoring', 'automatic deployment tools ( Terraform', 'Ansible', 'etc . )', 'cloud architectures ( vCloud', 'Azure', 'AWS )', 'monitoring tools', 'OpenShift', 'DockerHub', 'Kubernetes', 'CloudALM', 'Jira', 'Confluence', 'Git', 'Jenkins ( CloudBees )', 'Nexus', 'SonarHarbour', 'scripting languages ( Groovy', 'Python', 'JavaScript', 'etc . )', 'PostgreSQL', 'MongoDB', 'AzureCosmosDB']


# need evaluation

# Some adjustement to try 

In [601]:
processed_data.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
429     True
430    False
431    False
432     True
433    False
Length: 3464, dtype: bool

In [602]:
processed_data2=processed_data.drop_duplicates()

In [603]:
len(processed_data2)

2054

In [604]:
def word2vec_model2():
    w2v_model2 = Word2Vec(min_count=1,
                     window=3,
                     vector_size=50,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20)
    
    w2v_model2.build_vocab(processed_data2)
    w2v_model2.train(processed_data, total_examples=w2v_model.corpus_count, epochs=300, report_delay=1)
    
    return w2v_model2

In [605]:
w2v_model2 = word2vec_model2()
w2v_model2.save('word2vec_model_V2')

In [606]:
emb_vec2 = w2v_model2.wv

In [607]:
similarity(df2['skills'][4],df1['skill_required'][2],emb_vec2)#need to train the emb_vec on the skills dataset too

(0.17915858044855756,
 ['Collaboration',
  'architecture',
  'design',
  'deployment solutions',
  'system integration',
  'automation',
  'execution support',
  'pipeline deployments',
  'ALM tools',
  'cyber security',
  'SW deployments',
  'automation functional regression tests',
  'continuous integration',
  'environments infrastructures',
  'continuous monitoring',
  'automatic deployment tools ( Terraform',
  'Ansible',
  'etc . )',
  'cloud architectures ( vCloud',
  'Azure',
  'AWS )',
  'monitoring tools',
  'OpenShift',
  'DockerHub',
  'Kubernetes',
  'CloudALM',
  'Jira',
  'Confluence',
  'Git',
  'Jenkins ( CloudBees )',
  'Nexus',
  'SonarHarbour',
  'scripting languages ( Groovy',
  'Python',
  'JavaScript',
  'etc . )',
  'PostgreSQL',
  'MongoDB',
  'AzureCosmosDB'],
 [0.01228260892163513,
  0.024947112438300185,
  -0.16211852168848348,
  0.3082816611665612,
  -0.1371445855882149,
  0.1449384003796424,
  0.2305559876830418,
  0.23716890133905535,
  0.1004641328333793

first model gave 0.05 wich is more  logical than the 0.17 in the second model we'll go for the first one for now 

In [45]:
print(df1.columns)
print(df2.columns)

Index(['job_title', 'skill_required', 'location', 'link'], dtype='object')
Index(['URL', 'job_title', 'Name', 'skills', 'experiences'], dtype='object')


In [80]:
def match_person(cv_row,jbdf,emb_vec):
    sim_score=0
    for j in range(len(jbdf)):
        s=similarity(cv_row['skills'],jbdf['skill_required'][j] , emb_vec)[0]
        if s>sim_score:
            sim_score, skills_list, score_list = similarity(cv_row['skills'],jbdf['skill_required'][j] , emb_vec)
            row_job=jbdf.iloc[j]
    return row_job,sim_score,skills_list,score_list        
    
    

In [52]:
def match_job(jb_row,cvdf,emb_vec):
    sim_score=0
    for j in range(len(cvdf)):
        s=similarity(cvdf['skills'][j],jb_row['skill_required'] , emb_vec)[0]
        if s>sim_score:
            sim_score, skills_list, score_list = similarity(cvdf['skills'][j],jb_row['skill_required'] , emb_vec)
            row_job=cvdf.iloc[j]
    return row_job,sim_score,skills_list,score_list  
    

In [81]:
match_person(df2.iloc[1],df1,emb_vec)

C:\Users\MSI GF63\AppData\Local\Temp\ipykernel_7204\3122937574.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))


(job_title         Implementation process management APP Internship
 skill_required                                              [Java]
 location                                                       NaN
 link                                        DATAHORIZON-page17.pdf
 Name: 356, dtype: object,
 1.0,
 ['Java'],
 [1.0])

In [55]:
match_job(df1.iloc[3],df2,emb_vec)

C:\Users\MSI GF63\AppData\Local\Temp\ipykernel_7204\3122937574.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))


(URL                       https://www.linkedin.com/in/nourkobbi/
 job_title                   data science and engineering student
 Name                                                  Nour Kobbi
 skills         [angularjs net framework developpement logicie...
 experiences                                      nan:nan,nan:nan
 Name: 247, dtype: object,
 0.7036426325967947,
 ['Kafka',
  'Java8 ( JDK11',
  '17 )',
  'Spring Cloud Stream',
  'Spring Boot',
  'Spring Cloud ( Web',
  'Data',
  'JPA )',
  'Spring Framework Reactive',
  'WebFlux',
  'Microservices',
  'Event-Driven architecture',
  'MongoDB',
  'Neo4J',
  'SaaS software level',
  'Openshift',
  'K8S',
  'Flink'],
 [1.0000000000000002,
  1.0,
  1.0,
  0.3575618375810914,
  0.37840990834611477,
  1.0,
  0.9118453209603206,
  1.0,
  0.3575618375810914,
  0.7850050059549888,
  1.0,
  1.0,
  0.4689299917113523,
  0.6629063057116708,
  1.0,
  0.41201777520673816,
  0.630056372773615,
  0.9999999999999999,
  0.860062981346659,
  0

In [56]:
def Match(cvdf,jbdf,emb_vec):
    matchdf = pd.DataFrame(columns=['URL', 'Name', 'job_matched', 'job skills', 'total_score', 'score_list'])
    for i in range(len(cvdf)):
        print(cvdf.iloc[i])
        sim_score=0
        for j in range(len(jbdf)):
            s=similarity(cvdf['skills'][i],jbdf['skill_required'][j] , emb_vec)[0]
            if s>sim_score:
                sim_score, skills_list, score_list = similarity(cvdf['skills'][i],jbdf['skill_required'][j] , emb_vec)
                row_job=jbdf.iloc[j]
        matchdf = pd.concat([matchdf, pd.DataFrame({
                  'URL': [cvdf['URL'][i]], 
                  'Name': [cvdf['Name'][i]], 
                  'job_matched': [row_job['job_title']], 
                  'job skills': [skills_list], 
                  'total_score': [sim_score], 
                  'score_list': [score_list]
                  })])
            
    return matchdf    



         


In [73]:
matchdf=Match(df2,df1,emb_vec)

URL                        https://www.linkedin.com/in/talel-kb/
job_title            web developer  software engineering student
Name                                                Talel Kbaier
skills         [analyse donnees competences analytiques frame...
experiences    Développeur web:janv. 2023 - aujourd’hui · 2 m...
Name: 0, dtype: object


C:\Users\MSI GF63\AppData\Local\Temp\ipykernel_15404\3122937574.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))


URL                 https://www.linkedin.com/in/omar-talbi-sfax/
job_title                                freelance web developer
Name                                                  Omar Talbi
skills         [laravel react javascript php microsoft bot fr...
experiences    Freelance Web Developer:août 2019 - aujourd’hu...
Name: 1, dtype: object


C:\Users\MSI GF63\AppData\Local\Temp\ipykernel_15404\3122937574.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))


URL            https://www.linkedin.com/in/hassen-knani-21991...
job_title                             web developer chez zetabox
Name                                                Hassen Knani
skills         [management service client microsoft office ve...
experiences    React Js Instructor:mars 2022 - aujourd’hui · ...
Name: 2, dtype: object


...

C:\Users\MSI GF63\AppData\Local\Temp\ipykernel_15404\3122937574.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))


URL                    https://www.linkedin.com/in/hamza-samaiy/
job_title           data science master student  works at vermeg
Name                                                Hamza Samaiy
skills         [developpement web javascript php laravel feui...
experiences    Software Programmer:Jan 2023 - Present · 2 mos...
Name: 432, dtype: object


C:\Users\MSI GF63\AppData\Local\Temp\ipykernel_15404\3122937574.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))


URL            https://www.linkedin.com/in/zitouni-amal-286b9...
job_title      ingénieur en informatique  spécialité data sci...
Name                                                Zitouni Amal
skills         [microsoft azure machine learning apprentissag...
experiences    Ingénieur BI:Oct 2022 - Present · 5 mos,Member...
Name: 433, dtype: object


C:\Users\MSI GF63\AppData\Local\Temp\ipykernel_15404\3122937574.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))


In [75]:
print(matchdf.head())

                                                 URL           Name  \
0              https://www.linkedin.com/in/talel-kb/   Talel Kbaier   
0       https://www.linkedin.com/in/omar-talbi-sfax/     Omar Talbi   
0  https://www.linkedin.com/in/hassen-knani-21991...   Hassen Knani   
0          https://www.linkedin.com/in/safwendammak/  Safwen Dammak   
0  https://www.linkedin.com/in/oumayma-b%C3%A9hi-...   Oumayma Béhi   

                                         job_matched  \
0   Implementation process management APP Internship   
0   Implementation process management APP Internship   
0                Master Ingénieur réseau et sécurité   
0  CONCEPTION ET DÉVELOPPEMENT D'UN SIMULATEUR SO...   
0                              innovation specialist   

                              job skills  total_score  \
0                                 [Java]     1.000000   
0                                 [Java]     1.000000   
0                               [Projet]     1.000000   
0       

In [77]:
matchdf.to_excel('CSV\\Matched_data.xlsx', index=False)


## Pretrained Bert Model

In [104]:
pip install sentence-transformers

     -------------------------------------- 86.0/86.0 kB 805.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.2/1.2 MB 729.5 kB/s eta 0:00:00
     ------------------------------------ 977.5/977.5 kB 562.7 kB/s eta 0:00:00
     ------------------------------------ 172.3/172.3 MB 827.6 kB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125960 sha256=9162504e4d057cbf2a166882ed93f3d60c90d6f8fb87e63b56df1c441d433fe3
  Stored in directory: c:\users\msi gf63\appdata\local\pip\cache\wheels\0a\f5\dd\9d00836c4e9e279c2a59d5b0ab72dafa66cbc626a327c550dd
Successfully built sentence-transformers
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
Note: you may need to restart the kernel to use updated packages.


In [57]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [59]:
model.save('bertMODEL')

In [60]:
sentence_embeddings = model.encode(df2['skills'][1])


In [61]:
model.encode('python')

array([-2.46460795e-01,  4.16334391e-01,  1.09131372e+00, -6.30743802e-02,
        7.99204409e-02,  1.09192276e+00,  1.70364738e-01,  1.51926517e+00,
       -1.28249586e+00,  7.09121525e-01, -9.87488806e-01,  4.79984552e-01,
       -6.58217728e-01,  1.47507906e-01,  6.19223833e-01,  3.07309180e-02,
       -8.29079151e-01, -5.82966626e-01,  6.27305448e-01, -5.22945523e-02,
       -1.25515461e-02,  8.77614200e-01, -4.05389637e-01, -7.83491433e-01,
        5.16248763e-01, -1.69070423e-01,  1.81295976e-01, -2.95025080e-01,
       -6.98406935e-01,  1.28157631e-01, -7.00810432e-01, -4.91766959e-01,
        1.06496704e+00,  6.05641186e-01, -2.59486467e-01, -7.08003819e-01,
       -4.99629408e-01, -9.85539481e-02, -1.42428264e-01,  4.06365514e-01,
        1.42157876e+00, -4.02914166e-01,  4.64940071e-01,  3.83913040e-01,
       -6.28552973e-01,  2.37806633e-01,  5.89850664e-01,  3.72156143e-01,
       -4.50615764e-01, -7.06084967e-01, -9.19309437e-01, -4.05465275e-01,
        5.98991700e-02, -

In [62]:
def bert_similarity(sen1, sen2, model):
    p_sen1 = preprocess(sen1)
    p_sen2 = preprocess(sen2)
    sen_vec1 = model.encode(p_sen1).flatten()
    sen_vec2 = model.encode(p_sen2).flatten()
    return dot(sen_vec1, sen_vec2) / (norm(sen_vec1) * norm(sen_vec2))

In [63]:
def BERTsimilarity(skill,required,model):
    p_skill=preprocess(skill)
    p_required=preprocess(required)
    result=[]
    for i in range(len(p_required)):
        similarities=[]
        for j in range(len(p_skill)):
            similarities.append(bert_similarity(p_skill[j],p_required[i],model))
        result.append(max(similarities)) 
    result = [0 if pd.isna(x) else x for x in result]    
    if len(result) > 0:
        avg_similarity = sum(result) / len(result)
    else:
        avg_similarity = 0
        
    return avg_similarity,required,result    
    #we returned the similarity score , the required skills  ,all similiraty for every word alone

In [64]:
BERTsimilarity('Android,Java,Cloud','Java',model)

(1.0000001192092896, 'Java', [1.0000001])

In [65]:
BERTsimilarity('Java','Android,Java,Cloud',model)


(0.7070480684439341, 'Android,Java,Cloud', [0.67967814, 1.0000001, 0.44146594])

In [66]:
similarity('Java','Android,Java,Cloud',emb_vec)


(0.6702160629400611,
 'Android,Java,Cloud',
 [0.8247132502617905, 1.0, 0.185934938558393])

In [67]:
similarity('Java','cloud',emb_vec)

(0.185934938558393, 'cloud', [0.185934938558393])

In [76]:
BERTsimilarity('Java','cloud',model)

(0.44146594405174255, 'cloud', [0.44146594])

In [77]:
BERTsimilarity(df2['skills'][4],df1['skill_required'][2],model)

(0.7424085349948318,
 ['Collaboration',
  'architecture',
  'design',
  'deployment solutions',
  'system integration',
  'automation',
  'execution support',
  'pipeline deployments',
  'ALM tools',
  'cyber security',
  'SW deployments',
  'automation functional regression tests',
  'continuous integration',
  'environments infrastructures',
  'continuous monitoring',
  'automatic deployment tools ( Terraform',
  'Ansible',
  'etc . )',
  'cloud architectures ( vCloud',
  'Azure',
  'AWS )',
  'monitoring tools',
  'OpenShift',
  'DockerHub',
  'Kubernetes',
  'CloudALM',
  'Jira',
  'Confluence',
  'Git',
  'Jenkins ( CloudBees )',
  'Nexus',
  'SonarHarbour',
  'scripting languages ( Groovy',
  'Python',
  'JavaScript',
  'etc . )',
  'PostgreSQL',
  'MongoDB',
  'AzureCosmosDB'],
 [0.8266571,
  0.60174066,
  0.8234171,
  0.81405544,
  0.8387943,
  0.86670685,
  0.7367999,
  0.8197017,
  0.74529076,
  0.64547867,
  0.81405544,
  0.7248997,
  0.6007641,
  0.8300316,
  0.81405544,
  

In [71]:
def Bmatch_person(cv_row,jbdf,emb_vec):
    sim_score=0
    for j in range(len(jbdf)):
        s=BERTsimilarity(cv_row['skills'],jbdf['skill_required'][j] , emb_vec)[0]
        if s>sim_score:
            sim_score, skills_list, score_list = similarity(cv_row['skills'],jbdf['skill_required'][j] , emb_vec)
            row_job=jbdf.iloc[j]
    return row_job,sim_score,skills_list,score_list  

In [ ]:
Bmatch_person(df2.iloc[1],df1,model)

In [74]:
Bmatch_job(df1.iloc[3],df2,model)

ValueError: shapes (768,) and (0,) not aligned: 768 (dim 0) != 0 (dim 0)

In [73]:
def Bmatch_job(jb_row,cvdf,emb_vec):
    sim_score=0
    for j in range(len(cvdf)):
        s=BERTsimilarity(cvdf['skills'][j],jb_row['skill_required'] , emb_vec)[0]
        if s>sim_score:
            sim_score, skills_list, score_list = similarity(cvdf['skills'][j],jb_row['skill_required'] , emb_vec)
            row_job=cvdf.iloc[j]
    return row_job,sim_score,skills_list,score_list  

In [78]:
w2v_model = Word2Vec.load('word2vec_model_noSW')
